In [29]:
from cassandra import ConsistencyLevel
import json
import plotly.plotly as py
from plotly.graph_objs import *

# not used but might be useful for later
import cassandra
import sys
import random
import string
import getopt
import time
import datetime
import collections


In [30]:
# Connection Constants
KEYSPACE = 'amazon'
CONSISTENCY=ConsistencyLevel.LOCAL_ONE
SEED_NODES = ['127.0.0.1']
DATACENTER = None

# Cassandra Constants
META_COLUMN_FAMILY = 'metadata'

# Other Constants
X_COORD = []
Y_COORD = []

In [31]:
def connect(seeds, keyspace, datacenter=None, port=9042):
    from cassandra.io.libevreactor import LibevConnection
    from cassandra.cluster import Cluster
    from cassandra.policies import DCAwareRoundRobinPolicy, RetryPolicy, ExponentialReconnectionPolicy

    class CustomRetryPolicy(RetryPolicy):

        def on_write_timeout(self, query, consistency, write_type,
                             required_responses, received_responses, retry_num):

            # retry at most 5 times regardless of query type
            if retry_num >= 5:
                return (self.RETHROW, None)

            return (self.RETRY, consistency)


    load_balancing_policy = None
    if datacenter:
        # If you are using multiple datacenters it's important to use
        # the DCAwareRoundRobinPolicy. If not then the client will
        # make cross DC connections. This defaults to round robin
        # which means round robin across all nodes irrespective of
        # data center.
        load_balancing_policy = DCAwareRoundRobinPolicy(local_dc=datacenter)

    cluster = Cluster(contact_points=seeds,
                      port=port,
                      default_retry_policy=CustomRetryPolicy(),
                      reconnection_policy=ExponentialReconnectionPolicy(1, 60),
                      load_balancing_policy=load_balancing_policy,
                      protocol_version=3)

    cluster.connection_class = LibevConnection
    cluster.connection_class = LibevConnection
    cluster.control_connection_timeout = 10.0
    cluster.compression = False
    session = cluster.connect(keyspace)
    print 'Connection established with seed(s): %s at port: %s and keyspace: %s' %(seeds,port,keyspace)
    return session

In [32]:
def print_future_results(results):
    print results

In [33]:
def print_future_errors(errors):
    log.error(errors)

In [34]:
def solr_facet_async(connection):
    #Example using async queries 
    future_facet = connection.execute_async("""SELECT * FROM %s.%s WHERE solr_query='{"q":"title:Noir~", "facet":{"field":"categories"}}'  limit 10;"""% (KEYSPACE, META_COLUMN_FAMILY))
    future_callback = future_facet.add_callbacks(print_future_results, print_future_errors)
    print future_callback

In [35]:
def solr_facet(connection):
    facet = connection.execute("""SELECT * FROM %s.%s WHERE solr_query='{"q":"title:Noir~", "facet":{"field":"categories"}}'  limit 10;"""% (KEYSPACE, META_COLUMN_FAMILY))
    for rows in facet:
        i=rows.facet_fields
        s=json.loads(i)
        for r in s["categories"]:
            X_COORD.append(r)
            Y_COORD.append(s["categories"][r])
            print r, s["categories"][r]

In [36]:
def simple_query(connection):
    simple_read = connection.execute("""SELECT * FROM %s.%s limit 10;"""% (KEYSPACE, META_COLUMN_FAMILY))
    for rows in simple_read:
        print rows
    print (simple_read)

In [37]:
def plotly_facet_bar():
    data = Data([
    Bar(
        x=X_COORD,
        y=Y_COORD
    )
    ])
    plotly_url_bar = py.plot(data, filename='facet-bar')
    print plotly_url_bar


In [38]:
if __name__ == "__main__":
    # Create a single session multiple functions/casses can use
    connection = connect(SEED_NODES, keyspace=KEYSPACE, datacenter=DATACENTER)

    # Queries
    solr_facet(connection)
    #simple_query(connection)
    #solr_facet_async(connection)
    plotly_facet_bar()
    
    
    
    print 'fin!'

Connection established with seed(s): ['127.0.0.1'] at port: 9042 and keyspace: amazon
https://plot.ly/~MarcintheCloud/14
fin!


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:90: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.

